In [ ]:
#| default_exp InsuranceAI
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from insuranceaidl.utils import *

ModuleNotFoundError: No module named 'insuranceaidl'

# InsuranceAI

> Testing for Asymmetric Information in Insurance with Deep Learning

We read the data and we convert the variables to their correct types:

In [ ]:
#| export
data88 = read_data("j88")

## selecting  variables
We will work with a subset of the variables. Our endogeneous variables will be `Comprehensive coverage` for the choice of contract $y_1$ and `Not at fault` for the occurrence of a claim at fault $y_2$.
We also use the variable `Duration` to weight the observations, and 9 categorical covariates.

In [ ]:
#| export
y1, y2, w, X = select_variables(data88)

In [ ]:
X.describe()

In [ ]:
lhs_vars = pd.DataFrame({
    'Comprehensive coverage': y1, 
    'Claim at fault': y2, 'Duration': w
})
lhs_vars.describe()


The matrix of $(y_1,y_2)$ is as follows:

In [ ]:
pd.crosstab(lhs_vars['Comprehensive coverage'], lhs_vars['Claim at fault'])

In [ ]:
fig, ax = plt.subplots()
ax.hist(w, color='red', label='w', bins=20)
ax.set(xlabel="Proportion of the year covered", title="Weights")
ax.legend()
plt.show()



## first neural network
We fit a simple neural network to $y_1$, tentatively:

In [ ]:
model = keras.Sequential([
    layers.Dense(1, activation="sigmoid", input_dim=9),
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.fit(X.values[:3200,:], y1.values[:3200],epochs=5, batch_size=128)

In [ ]:
#| hide
import nbdev;nbdev.nbdev_export()